In [1]:
import pandas as pd
import numpy as np
from hamcrest import assert_that, equal_to

from er_evaluation.error_analysis import (
    record_error_table,
    expected_missing_links_from_table,
    expected_size_difference_from_table,
    expected_extra_links_from_table,
    cluster_sizes_from_table
)
from er_evaluation.summary import cluster_sizes
from er_evaluation.estimators import (
    pairwise_recall_design_estimate,
    pairwise_precision_design_estimate,
)
from er_evaluation.estimators._estimators import (
    ratio_estimator, 
    std_dev,
)

## Test two different precision/recall estimator computations are equal 

In [11]:
prediction = pd.Series(index=[1,2,3,4,5,6,7,8], data=[1,1,2,3,2,4,4,4])
reference = pd.Series(index=[1,2,3,4,5,6,7,8], data=["c1", "c1", "c1", "c2", "c2", "c3", "c3", "c4"])
sample = pd.Series(index=[1,2,3,4,5], data=["c1", "c1", "c1", "c2", "c2"])

for prediction, sample in [
    (prediction, sample),
    (sample, prediction),
    (prediction, reference),
    (reference, prediction),
    (prediction, prediction),
]:
    error_table = record_error_table(prediction, sample)
    cs = cluster_sizes_from_table(error_table)
    E_miss = expected_missing_links_from_table(error_table)
    E_size = expected_size_difference_from_table(error_table)
    weights = 1/cs

    N = cs*(cs - 1 - E_miss) * weights
    D = cs*(cs - 1 + E_size) * weights
    D1 = cs*(cs - 1) * weights

    assert_that(
        pairwise_precision_design_estimate(prediction, sample, weights="cluster_size"),
        equal_to(
            (ratio_estimator(N, D), std_dev(N, D))
        )
    )
    assert_that(
        pairwise_recall_design_estimate(prediction, sample, weights="cluster_size"),
        equal_to(
            (ratio_estimator(N, D1), std_dev(N, D1))
        )
    )

/home/olivieratduke/Desktop/Research/er-evaluation/er_evaluation/estimators.py:129: UserWarning: Some sample elements are not in the prediction.
  warnings.warn("Some sample elements are not in the prediction.")
/home/olivieratduke/Desktop/Research/er-evaluation/er_evaluation/estimators.py:129: UserWarning: Some sample elements are not in the prediction.
  warnings.warn("Some sample elements are not in the prediction.")
